In [50]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/ML_Project2/ML_Project2/scripts/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/ML_Project2/ML_Project2/scripts/'
/content/drive/MyDrive/ML_Project2/ML_Project2/scripts


In [51]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GroupKFold

## Custom libraries
import index_helpers as ih

### Import and prep data

In [52]:
## Import, index, and split

segmentation = True
fine_segmentation = True

# TAKE CARE: change of parameters for read and merge:
df = ih.read_and_merge_data(segmentation, fine_segmentation, exclude_expert=False)
df = ih.index_df_by_person(df)
df = ih.categorical_float_to_int(df)
df = ih.categorical_to_dummy(df)


In [56]:
#df1, df2, df3 = ih.separate_expert(df)
#df[df['Expert']==3.0]
#df1 = df[df['Expert']==1.0]
#df1.head()

In [ ]:
X_train, X_val, y_train, y_val = ih.train_test_split_on_index(features = df.drop("Label", axis=1),
                                                             label = df["Label"])

## Modify data for GroupKFold
groups = y_train.reset_index()['File_Name_split']
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

### Create pipeline and fit classifier

In [ ]:
from sklearn.svm import SVC

## Make pipeline - name classifier "clf"
clf_pipeline = Pipeline([("st_scaler", StandardScaler()),
                        ("clf", SVC())])

## Use "clf__" in order to correctly assign parameters to the clf object
clf_param_grid = {'clf__C': [0.1, 1, 10, 100, 1000],
                  'clf__tol': [0.0001], # bc it's not really useful to tune that param ?d
                  'clf__gamma': [0.1, 1, 10, 100],
                  'clf__degree':[0, 1, 2, 3, 4, 5, 6]}
## Instantiate GroupKFold to avoid data leakage - to be passed to cv
gkf=GroupKFold(n_splits=2)

## Set up Randomized search CV --> modulate n_iter for "quicker" results
clf_rand_auc = RandomizedSearchCV(estimator=clf_pipeline,
                                  param_distributions=clf_param_grid,
                                  cv=gkf, scoring='roc_auc', verbose=1, n_jobs=2, n_iter=50)

## Perform Group K-Cross-validation
clf_rand_auc.fit(X_train, y_train, groups=groups)

## Print results
print("Best score: ",  clf_rand_auc.best_score_)
print("Best estimator: ", clf_rand_auc.best_estimator_)

Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  3.9min
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:  8.4min finished


Best score:  0.5411889420261413
Best estimator:  Pipeline(memory=None,
         steps=[('st_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('clf',
                 SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=6,
                     gamma=0.1, kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.0001,
                     verbose=False))],
         verbose=False)
